In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import regex as re
import nltk
import openai
import torch
import openai
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from bertopic.representation import KeyBERTInspired, OpenAI
from hdbscan import HDBSCAN
from bertopic.dimensionality import BaseDimensionalityReduction
from sklearn.cluster import KMeans, AgglomerativeClustering
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from HanTa import HanoverTagger as ht
from langchain_text_splitters import RecursiveCharacterTextSplitter
from SpeechProcessing import documentImporter, dategetter
from Coalitions import scrape_german_coalitions


/home/pc/VSCode/Staging/Scripts/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
Abortion_corpus_df = pd.read_csv("./CSV/Corpus_Chunked_full_annotated_cleaned.csv", header = 0)
Abortion_corpus_df = Abortion_corpus_df[Abortion_corpus_df['qwen2.5:7b'].notna() & (Abortion_corpus_df['qwen2.5:7b'] == 1)]

In [3]:
# Store patterns in dictionaries for better organization
patterns = {
    "preamble": re.compile(r'("id":"\d{4}")(.*?)Uhr.{0,150}(Alterspräsidentin|Alterspräsident|Vizepräsidentin|Vizepräsident|Vizekanzlerin|Vizekanzler|Präsidentin|Präsident|Kanzlerin|Kanzler).{0,50}?:', re.DOTALL), #I swear this makes sense!
    "appendix": re.compile(r'(\(Schluß der Sitzung: \d+(.|:)\d+ Uhr.?\)|\\nAnlagen zum Stenographischen Bericht|\\nAnlage 1)(.*?)("id":"\d{4})', re.DOTALL),
    "appendix_last": re.compile(r'(\(Schluß der Sitzung: \d+(.|:)\d+ Uhr.?\)|\\nAnlagen zum Stenographischen Bericht|\\nAnlage 1)(.*?)', re.DOTALL),    # Last appendix without "id" at the end
    "party_speaker": re.compile(r'[^\s,]+ [^\s,]+ \([^\s,]+\)\s?:', re.DOTALL),                                                                         # Generic pattern for speeches, e.g. 'Speaker (Party) :'                                           
    "party_speaker_CDU": re.compile(r'[^\s,]+ [^\s,]+ \(CDU/CSU\)\s?:', re.DOTALL),                                                                     # Specific pattern for CDU speeches
    "party_speaker_FDP_random": re.compile(r'[^\s,]+ [^\s,]+ \(F.D.P.\)\s?:', re.DOTALL),                                                               # For an ungodly reason the FDP was briefly referred to as F.D.P. 1999-2000. I suspect this is a conspiracy to sabotage my thesis and social science in general.
    "party_speaker_new": re.compile(r'\[\w+\]\s?', re.DOTALL),                                                                                          # New pattern for speeches after 2013. The previous pattern 'Speaker (Party) :' was replaced with 'Speaker [Party] :' in the Bundestag protocol.
    "party_speaker_CDU_new": re.compile(r'\[CDU+/CSU\]\s?:', re.DOTALL),                                                                                # Specific pattern for CDU speeches
    "minister_speaker": re.compile(r'(?:[^\n,]+,\s+Bundesminister(?:in)?\s+(?:der|für|des)\s+[^\n:]+:)', re.DOTALL | re.UNICODE),                      # Ministers are usually addressed with 'Bundesminister der ... i.e. Finanzen'
    "chancellor_speaker": re.compile(r', (?:(?:Bundes|Vize)?[Kk]anzlerin?):', re.DOTALL),                                                                 # Chancellor speeches are usually addressed with 'Bundeskanzlerin:' or 'Vizekanzlerin:'
    "reactions": re.compile(r'\(\w\w+ (.*?)\)', re.DOTALL),                                                                                             # Reactions are usually in the form '(Applaus)', '(Beifall)', '(Zuruf)', these simple reactions are removed here
    "remarks": re.compile(r'\((?!CDU/CSU|CDU|CSU|SPD|FDP|F.D.P.|AfD|Die Linke|Bündnis 90/Die Grünen|Bündnis 90 / Die Grüne|Die Grünen|LINKE|PDS|Piraten|NPD|REP|DVU|ÖDP|Tierschutzpartei|MLPD|DKP|BP|SSW|Fraktionslos)[^(]*?:[^()]+\)', re.DOTALL) # Excludes party markers i.e. --> Joachim Gauck (CDU) : I need to keep these to identify individual speeches which
}

keywords = [
    "Schwangerschaftsabbruch",
    "Abtreibung",
    "abgetrieben",
    "Abtreibungsgesetz",
    "Abtreibungsrecht",
    "§ 218",
    "Schwangerschaft abgebrochen",
    "Schwangerschaft",
    "Schwanger",
    "Paragraf 218",
    "Schwangerschaftskonfliktgesetz",
    "Fristenlösung",
    "Indikationsregelung",
    "Beratungspflicht",
    "Beratungsschein",
    "werdendes Leben",
    "ungeborenes Kind",
    "ungeborenes Leben",
    "werdende Mutter",
    "Strafbarkeit Schwangerschaftsabbruch",
    "Entkriminalisierung",
    "Legalisierung",
    "Abbruchsversorgung",
    "medizinische Indikation",
    "kriminologische Indikation",
    "embryopathische Indikation",
    "Schwangerschaftskonfliktberatung",
    "Ärzt*innen Schwangerschaftsabbruch",
    "Arzt Schwangerschaftsabbruch",
    "Kostenübernahme Abtreibung",
    "Gesundheitsversorgung Schwangere",
    "Komplikationen Schwangerschaftsabbruch",
    "psychologische Betreuung",
    "Selbstbestimmungsrecht",
    "reproduktive Rechte",
    "reproduktive Selbstbestimmung",
    "Frauenrechte",
    "Stigmatisierung Abbruch",
    "Tabu Abtreibung",
    "Lebensschutz",
    "ungewollt schwanger",
    "Indikationslösung",
    "Fristenlösung",
    "Schutz des ungeborenen Lebens",
    "Diskriminierung Schwangerer",
    "Versorgung ungewollt Schwangerer",
    "Bundestagsdebatte Schwangerschaftsabbruch",
    "Parlamentsdebatte Abtreibung",
    "Gesetzesentwurf Schwangerschaftsabbruch",
    "Gesetzesänderung §218",
    "Antrag Schwangerschaftsabbruch",
    "Abstimmung Schwangerschaftsabbruch",
    "Expertenkommission Schwangerschaftsabbruch",
    "Öffentlichkeit Schwangerschaftsabbruch",
    "Gruppenantrag Abtreibung",
    "Ampel-Koalition Abtreibung",
    "CDU/CSU Position Abtreibung",
    "Liberalisierung Abtreibungsrecht",
    "Werbeverbot Schwangerschaftsabbruch",
    "Paragraf 219a",
    "Beratungsregel",
    "Kompromiss Schwangerschaftsabbruch",
    "Verfassungsgericht Urteil Schwangerschaftsabbruch",
    "Spätabbruch",
    "Minderjährige Schwangere",
    "Schwangere Jugendliche",
    "religiöse Verbände Abtreibung",
    "Statistik Schwangerschaftsabbruch",
    "Wir haben abgetrieben",
    "219a Werbeverbot",
    "Schwangerschaftsabbruch EU",
    "Internationaler Vergleich Abtreibungsrecht",
    "ungewollt schwanger",
    "ungeborenes Leben",
    "Aufhebung Werbeverbot",
    "Pro Familia",
    "SPD Schwangerschaftsabbruch",
    "Grüne Schwangerschaftsabbruch",
    "FDP Schwangerschaftsabbruch",
    "Linke Schwangerschaftsabbruch",
    "AfD Schwangerschaftsabbruch",
    "Abtreibungskonflikt",
    "Kindstötung", 
    "Memmingen"
]

In [4]:
text_splitter_pre2013 = RecursiveCharacterTextSplitter(
    
    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=True,

    separators=[
        patterns['party_speaker'].pattern,
        patterns['party_speaker_CDU'].pattern,
        patterns['chancellor_speaker'].pattern,
        patterns['minister_speaker'].pattern,
        patterns['party_speaker_FDP_random'].pattern, # Strange pattern that occurs in the parliamentary protocols around 1999 idk why. 
        r'[.!?]+\s+', 
        r'\n\n',     
        r'\n',       
        r'\s+',      
    ]
)

# The utilization of two splitter is necessary due to some structural changes in the protocols after 2013 as the patterns changed. Don't as me  why, I just work here.

text_splitter_post2013 = RecursiveCharacterTextSplitter(

    chunk_size=1000,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=True,
    keep_separator=True,
    
    separators=[
    patterns['chancellor_speaker'].pattern,
    patterns['minister_speaker'].pattern,
    patterns['party_speaker_CDU'].pattern,  
    patterns['party_speaker'].pattern,
    r'[.!?]+\s+', 
    r'\n\n',       
    r'\n',         
    r'\s+',       
    ]
)


In [5]:
nltk.download('punkt')

def split_into_sentences(text):
    """Split text into individual sentences using NLTK"""
    sentences = nltk.sent_tokenize(text, language='german')
    # Filter out very short sentences (likely fragments)
    return [sent.strip() for sent in sentences if len(sent.strip()) > 20]

# Modified splitting function
Abortion_corpus_df_split = []

for idx, row in Abortion_corpus_df.iterrows():
    date_str = str(row['date'])
    year_match = re.search(r'datetime\.date\((\d{4})', date_str)
    
    if year_match:
        year = int(year_match.group(1))
        
        # Use NLTK sentence tokenizer for precise sentence splitting
        sentences = split_into_sentences(row['chunk'])
        
        # Create new rows for each sentence
        for sentence in sentences:
            new_row = row.copy()
            new_row['chunk'] = sentence
            new_row['Year'] = year
            Abortion_corpus_df_split.append(new_row)

# Convert list to DataFrame
Abortion_corpus_df_split = pd.DataFrame(Abortion_corpus_df_split)
print(f"Original chunks: {len(Abortion_corpus_df)}")
print(f"Split sentences: {len(Abortion_corpus_df_split)}")
print(f"Average sentence length: {Abortion_corpus_df_split['chunk'].str.len().mean():.1f} characters")

[nltk_data] Downloading package punkt to /home/pc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Original chunks: 2245
Split sentences: 67286
Average sentence length: 137.8 characters


In [ ]:
tagger = ht.HanoverTagger('morphmodel_ger.pgz')

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('german'))

enhanced_parliamentary_stopwords = [
    # Basic parliamentary titles and roles
    "herr", "frau", "dr", "prof", "präsident", "präsidentin", "vizepräsident", "vizepräsidentin","männer", "frauen", "herren" , "damen"
    "abgeordneter", "abgeordnete", "abgeordneten", "kollege", "kollegin", "kollegen", "kolleginnen",
    "staatssekretär", "staatssekretärin", "bundesminister", "ministerin", "minister",
    
    # Parliamentary procedure terms
    "sitzung", "beratung", "ausschuss", "fraktion", "drucksache", "gesetzentwurf", "entwurf",
    "zusatzfrage", "zwischenfrage", "wort", "damen", "herren", "meine", "verehrten",
    "parl", "abs", "reform", "beschluss", "antrag", "vorlage",
    
    # Party abbreviations and names
    "afd", "spd", "cdu", "csu", "fdp", "grüne", "linke", "bündnis", "fraktion",
    "cdu csu", "cdu/csu", "csu fraktion", "spd fraktion",
    
    # Common parliamentary phrases
    "damen herren", "verehrten damen", "liebe", "vielen dank", "dank", "bitte",
    "ärztinnen ärzte", "kolleginnen kollegen", "meine damen", "sehr geehrte",
    
    # Time references and procedural terms
    "heute", "jahr", "donnerstag", "bonn", "berlin", "bundestag", "bundesrat",
    "sagen", "müssen", "gibt", "schon", "ja", "mehr", "mal", "wissen", "möchte",
    "treffen", "entscheiden", "stimmt", "woche", "seitdem", "entwicklung",
    
    # Generic procedural words
    "000", "dm", "millionen", "ausgaben", "auftreten", "bemerkung", "zugleich",
    "zweite", "größte", "aussehen", "spricht", "alltag", "wortlaut",
    
    # Specific names that appear frequently (from your results)
    "hans", "peter", "klein", "thorsten", "lieb", "axel", "müller", "marco", "buschmann",
    "kroll", "schlüter", "christoph", "kerstin", "engelhard", "funcke", "scheu", "schleicher",
    "schäfer", "andre", "detlef", "parr", "waigel", "winkelmeier", "becker", "ostman",
    "pfeifer", "neumeister", "bärbel", "bas", "katrin", "helling", "göring", "eckardt",
    "plahr", "wolfgang", "kubicki", "jäger", "burkhard", "kinkel", "hirsch", "wurbs",
    "schoeler",
    
    # Generic transition words and fillers
    "geht", "muß", "unserer", "gesagt", "sei", "auch", "schleicher", "warum",
    "hier", "diese", "dann", "noch", "über", "nach", "durch", "aber", "nicht",
    "eine", "einer", "einem", "eines", "dass", "sich", "werden", "wurde", "wird",
    "haben", "hatte", "sind", "war", "waren", "sowie", "bzw", "etc", "usw",
    
    # Parliamentary location references
    "bonn", "berlin", "deutschen", "bundestag", "bundesrat", "deutschland", "ländern",
    
    # Numbers and dates
    "2008", "02", "15", "1979", "1978", "1975", "218", "219a", "stgb",
    
    # Common verbs in parliamentary context
    "sagen", "müssen", "wollen", "können", "sollen", "mögen", "lassen", "machen",
    "gehen", "kommen", "stehen", "liegen", "setzen", "nehmen", "geben", "bringen"
]

sklearn_stopwords_de = list(stopwords.words('german'))

stopwords = sklearn_stopwords_de + enhanced_parliamentary_stopwords


def stop_word_removal(x):
    token = x.lower().split()  # Convert to lowercase first
    return ' '.join([w for w in token if not w in stopwords])

def preprocess(text):
    # Remove parliamentary titles and names
    text = re.sub(r'\b(herr|frau|dr|prof|abgeordneter?)\s+\w+', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b(vizepräsident|präsident|staatssekretär)\w*\b', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\b(beratung|drucksache|sitzung|ausschuss)\b', '', text, flags=re.IGNORECASE)
    
    # Clean text
    text = re.sub(r'[^\w\s]', ' ', text)  
    text = re.sub(r'\d+', ' ', text)    
    text = re.sub(r'\s+', ' ', text)      
    text = text.strip()                   
    text = re.sub(r'\b\w\b', '', text)  
    
    # Process tokens
    tokens = text.lower().split()
    meaningful_tokens = []
    
    for token in tokens:
        if (len(token) > 2 and 
            token not in enhanced_parliamentary_stopwords and 
            token not in sklearn_stopwords_de and
            not token.isdigit()):
            token = [lemma for (word,lemma,pos) in tagger.tag_sent(token.split())][0].lower() 
            meaningful_tokens.append(token)
    
    return ' '.join(meaningful_tokens)

Abortion_corpus_df_split.loc[:, 'preprocessed'] = Abortion_corpus_df_split['chunk'].apply(stop_word_removal)

[nltk_data] Downloading package stopwords to /home/pc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
abortion_keywords = [
    ["volkskörper", "bevölkerungswachstum", "volksgesundheit"],
    ["schutzlos", "hilflos", "schutzbedürftig", "schutzbedürftige", "schutzbedürftigen", "schutzbedürftigkeit"],
    ["eugenik", "kranke kinder", "erbschäden", "erbgut", "behinderung", "behindert", "beeinträchtigt", "erbkrank"],
    ["werdendes leben", "ungeborenes kind", "ungeborenes leben", "lebensschutz", "schutz werdenden lebens", "recht auf leben"],
    ["selbstbestimmungsrecht", "reproduktive selbstbestimmung", "frauenrechte", "patriarchat", "patriarchal", "patriarchalisch", "menschenwürde", "würde der frau"],
    ["medizinische indikation", "kriminologische indikation", "embryopathische indikation"],
    ["hilfsbedürftig", "entscheidungshilfe", "mädchen", "beratung","beratungsbedarf", "in not geraten", "notlagen", "frauen in not"],
    ["individuell", "selbstbestimmung", "psychologisch", "psychisch", "psychische gesundheit"],
    ["pfuscher", "kriminelle", "skrupellos", "ausnutzen", "abzocke", "männer", "partner", "vater"],
    ["liberalisierung", "abschaffung", "feminismus", "frauenbewegung", "frauen", "recht auf abtreibung", "patriarchat", "würde"],
    ["soziale gerechtigkeit", "ungerecht", "zugang", "klassen", "ungleich", "tourismus"],
    ["aufgabe", "staat", "staatlicher auftrag", "rolle", "aufgabe", "signal", "zeigen", "klarstellen"],
    ["pragmatismus", "pragmatisch", "praktisch", "lösungsorientiert", "zielorientiert", "funktioniert nicht", "nicht reduziert", "reduziert"],
    ["unfähig", "ungeeignet", "schlechte eltern", "vernachlässigung", "asozial"],
    ["zeugung", "aufklärung", "kompetenz", "förderung", "sexualerziehung", "verhütung", "verhindern"],
    ["soziales problem", "gesellschaftliche folgen", "gesellschaftliche auswirkungen", "gesellschaftliche konsequenzen", "gesellschaft"],
    ["osten", "ddr", "ostdeutsch", "ostdeutsche", "ostdeutschen", "ostfrau", "frauen im osten", "frauen in der ddr", "einigungsvertrag"],
    ["memmingen", "stern", "reportage", "ard", "zdf", "skandal"], 
    ["urteil", "rechtsprechung", "bundesverfassungsgericht", "bverfg", "karlsruhe", "verfassungsgericht", "richter", "rechtsprechung", "1975", "urteile", "verfassung", "grundgesetz"],
    ["christ", "christlich", "evangelisch", "katholisch", "bischof", "kirchen"], 
    ["schwangerschaftskonflikt", "konflikt", "rechtsgut", "rechtsgüter"],
    ["soziale hilfen", "rahmen", "kindergarten", "kinderbetreuung", "erziehungsgeld", "erziehungsurlaub"]
]

In [ ]:
theoretical_keywords = [
    ["bundesverfassungsgericht", "bverfg", "karlsruhe", "verfassungsgericht", "richter", "rechtsprechung", "urteile", "verfassung", "grundgesetz", "artikel 2", "urteil", "richter", "rechtsprechung"],
    ["eigenverantwortlich", "selbstbestimmt", "eigenverantwortung", "selbstbestimmung"],
    ["pflicht", "lebensschutz", "schutzpflicht", "staatliche schutzpflicht", "signalwirkung", "schutzbedürftig"],
    ["recht auf leben", "lebensrecht"]
]

In [ ]:
embedding_model = SentenceTransformer("all-mpnet-base-v2", device="cuda")  
documents = Abortion_corpus_df_split['preprocessed'].tolist()

torch.cuda.empty_cache() 
embeddings = embedding_model.encode(documents, batch_size=16)


In [ ]:

client = openai.OpenAI(
    base_url = 'http://localhost:11434/v1', 
    api_key='ollama', 
)

representation_model = {
    "mistral:7b": OpenAI(client, model='mistral:7b')
}

empty_dimensionality_model = BaseDimensionalityReduction()

cluster_model2 = KMeans(n_clusters=25)

cluster_model = AgglomerativeClustering(n_clusters=50)

hdbscan_model = HDBSCAN(
    min_cluster_size=10,  
    min_samples=3, 
    metric='euclidean', 
    cluster_selection_method='eom',
    prediction_data=True
)

ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)


topic_model = BERTopic(
    language="german",
    nr_topics="auto",
    umap_model=empty_dimensionality_model,
    hdbscan_model=cluster_model2,
    embedding_model= embedding_model,
    representation_model=representation_model,
    seed_topic_list=abortion_keywords,
    ctfidf_model=ctfidf_model,
    calculate_probabilities=True)


vectorizer_model_de = CountVectorizer(stop_words=stopwords)
topics = topic_model.fit_transform(documents, embeddings)
topic_model.update_topics(documents)

In [ ]:
res = topic_model.get_topic_info()
print(topic_model.get_topic_info())


In [ ]:
cols = ['Topic', 'Name', 'Representation', 'mistral:7b', 'Representative_Docs']
df_tex = res[cols].copy()

def stringify(cell):
    if isinstance(cell, (list, tuple)):
        return ', '.join(str(i) for i in cell)
    return str(cell)

for c in df_tex.columns:
    df_tex[c] = df_tex[c].apply(stringify)

out_path = "./Outputs/topics_representation.tex"
df_tex.to_latex(out_path, index=False, escape=False, longtable=True)

print(f"Wrote LaTeX table to: {out_path}")
print(df_tex.head().to_string(index=False))

In [ ]:
topic_model.visualize_barchart()

if not hasattr(topic_model, "topic_labels_"):
	topic_model.topic_labels_ = {}

# Visualize with the updated custom labels
topic_model.visualize_topics()
topic_model.visualize_hierarchy(custom_labels=True)

In [ ]:
topics_over_time = topic_model.topics_over_time(documents, timestamps)
topics_over_time['Timestamp'] = pd.to_datetime(topics_over_time['Timestamp'])
topic_info = topic_model.get_topic_info()

fig = topic_model.visualize_topics_over_time(topics_over_time, normalize_frequency=True,custom_labels=True)
fig.show()

In [7]:
def BERTopicer (Corpus = Abortion_corpus_df_split, start_year = 1950, end_year =1980):
    year_mask = (Corpus['Year'] <= end_year) & (Corpus['Year'] >= start_year)
    filtered_df = Corpus.loc[year_mask]
    Docs = filtered_df['preprocessed'].tolist()
    topic_model = BERTopic(verbose=False).fit(Docs)
    print(f"BERTopic Model for years {start_year} to {end_year}:")
    print(topic_model.get_topic_info())
    return topic_model

In [8]:
ranges = [(2011,2024), (1991,2010), (1981,1990), (1950,1980)]
topic_models = {f"{s}-{e}": BERTopicer(Corpus=Abortion_corpus_df_split, start_year=s, end_year=e)
                for s,e in ranges}

BERTopic Model for years 2011 to 2024:
     Topic  Count                                               Name  \
0       -1   3481                           -1_ist_ganz_werden_immer   
1        0    636  0_schwangerschaft_schwangerschaftsabbruch_schw...   
2        1    265  1_bundesregierung_bundesverfassungsgericht_bun...   
3        2    189  2_informationen_information_informieren_inform...   
4        3    188  3_gesellschaft_gesellschaftlichen_gesellschaft...   
..     ...    ...                                                ...   
180    179     11  179_juristische_juristen_juristinnen_grundlage...   
181    180     11                            180_gyde_jensen_fdp_tue   
182    181     11                181_gewesen_wre_miteinzubinden_gang   
183    182     10  182_untersttzung_energiewende_frhchen_unbildun...   
184    183     10                183_geehrten_herren_abgelehnt_hilft   

                                        Representation  \
0    [ist, ganz, werden, immer, knnen,

In [10]:
df_2011_2024 = topic_models['2011-2024'].get_topic_info().head(10)
df_1991_2010 = topic_models['1991-2010'].get_topic_info().head(10)
df_1981_1990 = topic_models['1981-1990'].get_topic_info().head(10)
df_1950_1980 = topic_models['1950-1980'].get_topic_info().head(10)

In [14]:
def df_to_latex_rows(df):
    rows = []
    for _, row in df.iterrows():
        topic = row["Topic"]
        count = row["Count"]
        name = row["Name"]
        rep = ", ".join(row["Representation"])
        docs = ", ".join(row["Representative_Docs"])
        line = f"{topic} & {count} & {name} & [{rep}] & [{docs}] \\\\"
        rows.append(line)
    return "\n".join(rows)

print(df_to_latex_rows(df_1950_1980))


-1 & 3892 & -1_leben_frage_werden_ist & [leben, frage, werden, ist, haben, wird, schutz, kann, lebens, mutter] & [sie, blüm, gestern — bleiben moment —, strafdrohung staat pflicht genommen, sozialen maßnahmen ergreifen, notwendig sind, problem wirklich lösen — glauben nicht, problem strafrechtlich lösen —, genauso eyrich sache total umgekehrt angesehen., vorschlag, gerade schutz werdenden lebens ernst nehmen., frage sie, herren, debatte einfach tun können, lage, unerwünschte schwangerschaft hat, lage, frage ganz persönlich stellen kann, entspricht, grundsatz, soeben genannt habe, vergleichen können.] \\
0 & 742 & 0_schwangerschaftsabbruch_schwangerschaft_schwangere_schwangeren & [schwangerschaftsabbruch, schwangerschaft, schwangere, schwangeren, schwangerschaftsabbrche, schwangerschaftsabbruchs, abbruch, arzt, schwangerschaften, fortsetzung] & [§ d abs. 1 nr. 1 darf schwangerschaft erst abgebrochen werden, nachdem — zitiere — „die schwangere ... behördlich ermächtigte beratungsstelle v